In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

In [3]:
print(tf.__version__)

2.5.0


In [4]:
import pathlib

data_dir = pathlib.Path('drive/My Drive/Soy/Data')

In [5]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

5659


In [6]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'), shuffle=False)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)

In [7]:
for f in list_ds.take(5):
  print(f.numpy())

b'drive/My Drive/Soy/Data/Copper_phytotoxicity/fito289.jpg'
b'drive/My Drive/Soy/Data/Bacterial_blight/creb193.jpg'
b'drive/My Drive/Soy/Data/Copper_phytotoxicity/fito307.jpg'
b'drive/My Drive/Soy/Data/Brown_spot/sept621.jpg'
b'drive/My Drive/Soy/Data/Southern_Blight/malv879.jpg'


In [8]:
class_names = np.array(sorted([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"]))
print(class_names)

['Bacterial_blight' 'Brown_spot' 'Copper_phytotoxicity' 'Downy_mildew'
 'Healthy' 'Mosaic_virus' 'Powdery_mildew' 'Rust' 'Southern_Blight']


In [9]:
val_size = int(image_count * 0.2)
train_ds = list_ds.skip(val_size)
val_ds = list_ds.take(val_size)

In [10]:
print(tf.data.experimental.cardinality(train_ds).numpy())
print(tf.data.experimental.cardinality(val_ds).numpy())

4528
1131


In [11]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)

In [12]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # resize the image to the desired size
  return tf.image.resize(img, [img_height, img_width])

In [13]:
def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [14]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

batch_size = 8
img_height = 256    
img_width = 256

In [15]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [16]:
for image, label in train_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

Image shape:  (256, 256, 3)
Label:  7


In [17]:
def configure_for_performance(ds):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size=1000)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)



In [18]:
from tensorflow.keras import layers

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [19]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image)) 


0.0 1.0


# MODELO VGG19

In [20]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [21]:
# load model and specify a new input shape for images and avg pooling output
new_input = tf.keras.layers.Input(shape=(256, 256, 3))


# load model without classifier layers
model2 = tf.keras.applications.VGG19(include_top=False, input_tensor=new_input, weights='imagenet')
model2.trainable = True
# add new classifier layers
flat1 = Flatten()(model2.layers[-1].output)
class1 = Dense(128, activation='relu')(flat1)
output = Dense(9, activation='softmax')(class1)
# define new model
model2 = Model(inputs=model2.inputs, outputs=output)
# summarize
model2.summary()

80142336/80134624 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
______________________________________________________________

In [22]:
base_learning_rate = 0.001
model2.compile(optimizer=tf.keras.optimizers.SGD(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
history2 = model2.fit(train_ds,
                    validation_data=(val_ds),  
                    epochs=50
)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4930: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


566/566 [==============================] - 536s 884ms/step - loss: 1.5524 - accuracy: 0.5053 - val_loss: 0.8894 - val_accuracy: 0.6649
Epoch 2/50
566/566 [==============================] - 121s 213ms/step - loss: 0.6536 - accuracy: 0.7652 - val_loss: 0.5678 - val_accuracy: 0.7913
Epoch 3/50
566/566 [==============================] - 126s 223ms/step - loss: 0.4288 - accuracy: 0.8421 - val_loss: 0.5475 - val_accuracy: 0.8134
Epoch 4/50
566/566 [==============================] - 126s 223ms/step - loss: 0.3212 - accuracy: 0.8878 - val_loss: 0.4315 - val_accuracy: 0.8603
Epoch 5/50
566/566 [==============================] - 127s 224ms/step - loss: 0.2202 - accuracy: 0.9189 - val_loss: 0.4248 - val_accuracy: 0.8576
Epoch 6/50
566/566 [==============================] - 127s 224ms/step - loss: 0.1828 - accuracy: 0.9379 - val_loss: 0.4263 - val_accuracy: 0.8727
Epoch 7/50
566/566 [==============================] - 127s 224ms/step - loss: 0.1389 - accuracy: 0.9532 - val_loss: 0.4104 - val_accura

# Modelo ResNet 50

In [ ]:
import tensorflow as tf

In [ ]:
# load model and specify a new input shape for images and avg pooling output
new_input = tf.keras.layers.Input(shape=(256, 256, 3))

# load model without classifier layers
model1 = tf.keras.applications.ResNet50(include_top=False, input_tensor=new_input, weights='imagenet')
model1.trainable = True
# add new classifier layers
flat1 = Flatten()(model1.layers[-1].output)
class1 = Dense(128, activation='relu')(flat1)
output = Dense(9, activation='softmax')(class1)
# define new model
model1 = Model(inputs=model1.inputs, outputs=output)
# summarize
model1.summary()

In [ ]:
base_learning_rate = 0.001
model1.compile(optimizer=tf.keras.optimizers.SGD(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model1.fit(train_ds,
                    validation_data=(val_ds),  
                    epochs=50
)

# DenseNet201

In [ ]:
# load model and specify a new input shape for images and avg pooling output
new_input = tf.keras.layers.Input(shape=(256, 256, 3))

# load model without classifier layers
model3 = tf.keras.applications.DenseNet201(include_top=False, input_tensor=new_input, weights='imagenet')
model3.trainable = True
# add new classifier layers
flat1 = Flatten()(model3.layers[-1].output)
class1 = Dense(128, activation='relu')(flat1)
output = Dense(9, activation='softmax')(class1)
# define new model
model3 = Model(inputs=model3.inputs, outputs=output)
# summarize
model3.summary()

In [ ]:
base_learning_rate = 0.001
model3.compile(optimizer=tf.keras.optimizers.SGD(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history3 = model3.fit(train_ds,
                    validation_data=(val_ds),  
                    epochs=50
)

In [ ]:
#class_names = np.array(sorted([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"]))

labels = '\n'.join(class_names)

with open('labels.txt', 'w') as f:
  f.write(labels)

In [ ]:
saved_model_dir = 'drive/My Drive/Soy/'
tf.saved_model.save(model3, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
from google.colab import files

files.download('model.tflite')
files.download('labels.txt')

# EfficientNetB7

In [ ]:
# load model and specify a new input shape for images and avg pooling output
new_input = tf.keras.layers.Input(shape=(256, 256, 3))

# load model without classifier layers
model4 = tf.keras.applications.EfficientNetB7(include_top=False, input_tensor=new_input, weights='imagenet')
model4.trainable = True
# add new classifier layers
flat1 = Flatten()(model4.layers[-1].output)
class1 = Dense(128, activation='relu')(flat1)
output = Dense(9, activation='softmax')(class1)
# define new model
model4 = Model(inputs=model4.inputs, outputs=output)
# summarize
model4.summary()

In [ ]:
base_learning_rate = 0.001
model4.compile(optimizer=tf.keras.optimizers.SGD(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history4 = model4.fit(train_ds,
                    validation_data=(val_ds),  
                    epochs=50
)

# Meu Modelo

In [ ]:

num_classes = 3

model5 = tf.keras.Sequential([
  
  # layers.experimental.preprocessing.Rescaling(1./255),
   
   #_______________________ Parte Convolucional
   tf.keras.layers.Conv2D(128, 3, activation='relu'),
   tf.keras.layers.MaxPooling2D(),
   tf.keras.layers.Conv2D(64, 3, activation='relu'),
   tf.keras.layers.MaxPooling2D(),
   tf.keras.layers.Conv2D(32, 3, activation='relu'),
   tf.keras.layers.MaxPooling2D(),
  #_______________________ Rede Neural
   tf.keras.layers.Flatten(),
   tf.keras.layers.Dense(128, activation='relu'),
   tf.keras.layers.Dense(num_classes, activation='softmax')
 ])

In [ ]:

base_learning_rate = 0.001
model5.compile(optimizer=tf.keras.optimizers.SGD(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history5 = model5.fit(train_ds,
                    validation_data=(val_ds),  
                    epochs=20
)

In [ ]:
model5.summary()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# Gráfico de perda (loss) de validação:

In [ ]:

plt.plot(history.history['val_loss'])
plt.plot(history2.history['val_loss'])
plt.plot(history3.history['val_loss'])
plt.plot(history4.history['val_loss'])
#plt.plot(history5.history['val_loss'])
plt.title('Validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['ResNet50', 'VGG19', 'DenseNet201', 'EfficientNetB7'], loc='upper right')
plt.show()

# Gráfico de perda (loss) de treinamento:

In [ ]:

plt.plot(history.history['loss'])
plt.plot(history2.history['loss'])
plt.plot(history3.history['loss'])
plt.plot(history4.history['loss'])
#plt.plot(history5.history['loss'])
plt.title('Train loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['ResNet50', 'VGG19', 'DenseNet201', 'EfficientNetB7'], loc='upper right')
plt.show()

# Gráfico de Accuracy de validação:

In [ ]:
# ver val_ds para conferir labels
plt.plot(history.history['val_accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.plot(history3.history['val_accuracy'])
plt.plot(history4.history['val_accuracy'])
#plt.plot(history5.history['val_accuracy'])
plt.title('Validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['ResNet50', 'VGG19', 'DenseNet201', 'EfficientNetB7'], loc='upper left')
plt.show()

# Gráfico de Accuracy de treino:

In [ ]:
# ver val_ds para conferir labels
plt.plot(history.history['accuracy'])
plt.plot(history2.history['accuracy'])
plt.plot(history3.history['accuracy'])
plt.plot(history4.history['accuracy'])
#plt.plot(history5.history['accuracy'])
plt.title('Train accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['ResNet50', 'VGG19', 'DenseNet201', 'EfficientNetB7'], loc='upper left')
plt.show()

# previsões do modelo para o Dataset de Validação

In [ ]:
list_val = list(val_ds)

In [ ]:
len(list_val)

In [ ]:
y = np.concatenate([y for x, y in list_val])
x = np.concatenate([x for x, y in list_val])

In [ ]:
x.shape

In [ ]:
import time

In [ ]:
ti = time.time()

predictions = model3.predict(x)
y_pred = np.argmax(predictions, axis=1)

tf = time.time()
elapsed = tf-ti 

In [ ]:
elapsed/1131

In [ ]:
predictions.shape

In [ ]:
y_pred = np.argmax(predictions, axis=1)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix

CM = confusion_matrix(y, y_pred)
print(CM)
print('\nTest Accuracy: ' + str(np.sum(np.diag(CM)) / np.sum(CM)))

# Colocar a Matriz de forma gráfica:

In [ ]:
import seaborn
import matplotlib
import matplotlib.pyplot as plt

fig_dims = (15, 10)
fig, ax = plt.subplots(figsize=fig_dims)

seaborn.heatmap(CM, annot=True, ax=ax,fmt="d", xticklabels=class_names, yticklabels=class_names, cmap="YlGnBu")
plt.show()